# Basics of network analysis 

In [ ]:
import networkx as nx
import seaborn as sns

In [ ]:
%pylab inline

## Connectivity and clustering of a graph

We study the network of coauthorships of Astro-Ph, from the SNAP database.

In [ ]:
filepath = "./../datasets/ca-AstroPh.txt"

In [ ]:
G = nx.Graph()

In [ ]:
fh = open(filepath, "r")
for line in fh.readlines():
    s = line.strip().split()
    if s[0] != "#":
        origin = int(s[0])
        dest = int(s[1])
        G.add_edge(origin, dest)
fh.close()

In [ ]:
print("The graph has", len(G), "nodes and", len(G.edges()), "edges")

In [ ]:
print("Is the graph simply connected?", nx.is_connected(G))

### Show the components of the graph

In [ ]:
print("The graph has", nx.number_connected_components(G), "connected components")

In [ ]:
for k in nx.connected_components(G):
    print(len(k))

### Extract the largest Connected Component as a subgraph

In [ ]:
nx.connected_components(G)

In [ ]:
graphs = list(nx.connected_components(G))

In [ ]:
H = G.subgraph(graphs[0])

In [ ]:
len(H)

In [ ]:
print(len(G) - len(H))

In [ ]:
print("Check that the graph is now connected")
nx.is_connected(H)

## Global clustering coefficient

The global clustering coefficient measures the number of triangles in the network and it's defined as: 
<center>
$C_\Delta = \frac{3 \times \text{triangles}}{\text{triplets}}$
</center> 

In [ ]:
nx.triangles(H)

How many triangles are there in the whole network?

In [ ]:
tt = sum(list(nx.triangles(H).values()))

In [ ]:
tt / 3

The transitivity is the fraction of all possible triangles in the network.

In [ ]:
nx.transitivity(H)

## Average clustering coefficient

 <img src="figure/clust_coeff.png" width="75%">
 

As an alternative to the global clustering coefficient, the overall level of clustering in a network is measured by Watts and Strogatz as the average of the local clustering coefficients of all the vertices $n$:

<center>
$\bar{C} = \frac{1}{n}\sum_{i=1}^{n} C_i.$
</center>

The clustering coefficient essentially quantifies how ones "friends" are "friends" with each other.


In [ ]:
print("The average clustering coefficient of H is")
nx.average_clustering(H)

## Average sorthest path length
### Warning! Calculating the shortest paths is intensive!! 

The graph is small world.

In [ ]:
nx.average_shortest_path_length(H)

In [ ]:
math.log(len(H), 10)

# Create an arbitrary graph G

In [ ]:
G = nx.Graph()

G.add_edges_from([(0,1),(0,4),(0,5),(1,3),(1,2),
                  (1,4), (4,6), (4,7), (4,8), (4,9), 
                  (3,10), (10,11), (10,12), (12,13), 
                  (12,14), (12,15), (13,16), (16, 17), 
                  (16, 18), (8,19), (19, 20), (19,21), 
                  (19,22), (8,23), (8,24), (23,24)])
pos = nx.fruchterman_reingold_layout(G);

plt.figure(figsize=(8,8));
plt.axis("off");
nx.draw_networkx_nodes(G, pos, node_size=300, node_color="lightgreen");
nx.draw_networkx_edges(G, pos, alpha=0.500);
nx.draw_networkx_labels(G, pos, font_color='black');
plt.show();

Observe how the central nodes vary depending on the measure we're using.

In [ ]:
fig = plt.figure(figsize=(20,15));

centralities = [list(nx.degree_centrality(G).values()), 
                list(nx.closeness_centrality(G).values()), 
                list(nx.betweenness_centrality(G).values()),
                list(nx.eigenvector_centrality(G).values())]
titles = ['Degree Centrality', 'Closeness Centrality', 
          'Betweenness Centrality', 'Eigenvector Centrality']

for i in range(4):
    ax = fig.add_subplot(2, 2, i+1);
    nc = nx.draw_networkx_nodes(G, pos, node_size=300, cmap=plt.cm.RdYlBu_r,
                            node_color=centralities[i]);
    nx.draw_networkx_edges(G, pos, alpha=0.500);
    nx.draw_networkx_labels(G, pos, font_color='white');
    plt.title(titles[i]);
    plt.axis('off');
    plt.colorbar(nc);

plt.show();

# The Zachary Karate club

This dataset, Zachary's Karate Club ([Zachary, 1977](https://www.jstor.org/stable/3629752)), is historically so important in network science, that NetworkX has even a function to import it! 

In [ ]:
K = nx.karate_club_graph()

In [ ]:
pos = nx.fruchterman_reingold_layout(K);
plt.figure(figsize=(8,8));
plt.axis("off");
nx.draw_networkx_nodes(K, pos, node_size=300, node_color="black");
nx.draw_networkx_edges(K, pos, alpha=0.500);
nx.draw_networkx_labels(K, pos, font_color="white");


plt.show();

## Clustering functions 

We write a function that implements the Girvan-Newman algorithm.



In [ ]:
def updateGraph1(G):

    ebw = nc.edge_betweenness(G)
    
    maxs = 0
    for k, v in ebw.items():
        if maxs < v:
            medge, maxs = k, v

    G.remove_edge(medge[0], medge[1])

In [ ]:
while nx.is_connected(G):

    # we remove links until the graph is connected
    updateGraph1(G)

In [ ]:
nx.is_connected(G)

In [ ]:
communities = [i for i in nx.connected_components(G)]
communities

In [ ]:
color_community = []

for i in range(0, len(G)):

    if i in communities[0]:
        color_community.append(0)
    else:
        color_community.append(1)

In [ ]:
plt.figure(figsize=(10, 8))

nx.draw_networkx(
    K,
    pos,
    node_color=color_community,
    cmap=plt.cm.RdBu,
    with_labels=True,
    label_color='white'
)